In [50]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
from matplotlib import pyplot as plt
import numpy as np
import math
import json
import os
from copy import deepcopy, copy
from collections import Counter

logging.basicConfig(
    format="[%(asctime)s] %(levelname)s: %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

In [51]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [52]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
        self._sticks = sum(self._rows)

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    def __eq__(self, __o: object) -> bool:
        return Counter(self._rows) == Counter(__o._rows)
        
    def __hash__(self) -> int:
        return hash(tuple(self._rows))
        
    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    @property
    def sticks(self) -> int:
        return self._sticks

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [53]:
def evaluate_state(state: Nim, player: int) -> int:
    if state:
        return 0
    else:
        if player == 0:
            return 1
        else:
            return -1

In [54]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim, complete=False) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["completation"] = sum(state.rows)/state.sticks
    if complete:
        brute_force = list()
        cooked["nim_sum"] = nim_sum(state)
        for m in cooked["possible_moves"]:
            tmp = deepcopy(state)
            tmp.nimming(m)
            brute_force.append((m, nim_sum(tmp)))
        cooked["brute_force"] = brute_force
    return cooked

In [55]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state, complete = True)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]


In [56]:
from functools import cache, lru_cache
MAX_DEPTH = 30

@lru_cache
def minmax(current_depth: int, state: Nim, player: int, alpha = -1, beta = 1) -> int:
    if MAX_DEPTH:
        if current_depth > MAX_DEPTH:
            return None, 0  
    value = evaluate_state(state, player)
    # print(f"Evaluating state {state}: {value}")
    if value != 0:
        return None, value
    else: 
        evaluations = list()
        data = cook_status(state)
        ps = sorted(data['possible_moves'], key=lambda k: -state.rows[k[0]])
        # print(f"Possible moves: {ps}")
        for ply in ps:
            new_nim = deepcopy(state)
            # print(f"Before: {new_nim}")
            new_nim.nimming(ply)
            # print(f"After: {new_nim}")
            _, val = minmax(current_depth + 1, new_nim, 1 - player, alpha, beta)
            evaluations.append((ply, val))
            if player:
                alpha = max(alpha, val)
            else:
                beta = min(beta, val)
            if beta <= alpha:
                break
    # print(f"Final evaluations: {evaluations}")
    return (max if player else min)(evaluations, key=lambda k: k[1])

In [57]:
def aggressive(state: Nim) -> Nimply:
    """Pick always the entire row if the number of active rows is odd"""
    data = cook_status(state)
    if data['active_rows_number'] % 2 == 0:
        # random move
        row, num_objects = random.choice(data['possible_moves'])
    else:
        # aggressive move
        row = data['longest_row']
        num_objects = state.rows[row]
    return Nimply(row, num_objects)

In [58]:
def minmax_strategy(state: Nim) -> Nimply:
    best_ply, eval = minmax(0, state, 0)
    # print(f"{best_ply} -> {eval}")
    return Nimply(best_ply[0], best_ply[1])

In [59]:
strategy = (minmax_strategy, optimal_strategy)

nim = Nim(5)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.debug(f"{player}: {ply}")
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

KeyboardInterrupt: 